In [ ]:
import findspark

In [ ]:
findspark.init("/home/nahle/spark-2.1.0-bin-hadoop2.7")

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("ML").getOrCreate()

# Bike Rental DataSet from UCI Machine Learning Repository
## Citations
Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelber
## Attributes on original data


- season : season (1:springer, 2:summer, 3:fall, 4:winter)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from [Web Link])
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
- weathersit : 
 - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
 - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
 - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
 - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)


## URL:
https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset

In [ ]:
rowData = spark.read.csv("Bike Rental UCI dataset.csv", inferSchema=True, header = True)

In [ ]:
rowData.show(n=5)

In [ ]:
rowData.printSchema()

In [ ]:
rowData.groupBy('dayOfWeek').count().show()

In [ ]:
rowData.groupBy('mnth').count().show()

In [ ]:
rowData.cache()

In [ ]:
rowData.groupBy('yr').count().show()

In [ ]:
rowData.groupBy('season').count().show()

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol='dayOfWeek', outputCol='day_cat')

In [ ]:
indexed_data =indexer.fit(rowData).transform(rowData)

In [ ]:
indexed_data.show()

In [ ]:
indexed_data.select('day_cat').distinct().orderBy('day_cat').show()

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [ ]:
indexed_data.columns


In [ ]:
vec = VectorAssembler(
  inputCols= [
    'season',
    'yr',
    'mnth',
    'hr',
    'holiday',
    'workingday',
    'weathersit',
    'temp',
    'hum',
    'windspeed',
    'days',
    'day_cat'
    ],
   outputCol = 'features'                  
 )

In [ ]:
data = vec.transform(indexed_data)

In [ ]:
data.show(truncate=False)

In [ ]:
for item in data.take(1)[0]:
    print (item)

In [ ]:
for item in data.take(3):
  print (item)
  print('\n')

In [ ]:
modelData = data.select('features', 'demand') 

In [ ]:
modelData.show(truncate =False)

In [ ]:
trainData, testData = modelData.randomSplit([0.7, 0.3])

In [ ]:
modelData.describe().show()

In [ ]:
trainData.describe().show()

In [ ]:
testData.describe().show()

In [ ]:
lr = LinearRegression(featuresCol='features', labelCol='demand', maxIter=1000)

In [ ]:
lr.explainParam('elasticNetParam')

In [ ]:
lr.explainParams()

In [ ]:
trainData.cache()

In [ ]:
testData.cache()

In [ ]:
lrModel = lr.fit(trainData)

In [ ]:
summary = lrModel.summary

In [ ]:
summary.explainedVariance

In [ ]:
summary.meanAbsoluteError

In [ ]:
data.select('demand').describe().show()


In [ ]:
summary.r2

In [ ]:
summary.predictions.show(n=3, truncate = False)

In [ ]:
print ("explainedVariance={}".format(summary.explainedVariance))
print ("meanAbsoluteError=%g" %summary.meanAbsoluteError)

In [ ]:
testResults = lrModel.evaluate(testData)

In [ ]:
testResults.residuals.show(n=10)

In [ ]:
testResults.residuals.groupBy().avg().show() 

- The average of the residuals does not reflect the reality as the residuals can be negative
- The mean absolute error is the average of the absolute values of the residuals

In [ ]:
from pyspark.sql.functions import abs
df= testResults.residuals
df.select(abs(df.residuals)).groupBy().avg().show()

In [ ]:
print ("r2=%g"%testResults.r2)   # my model explains x % of the variance of the data
print ("rootMeanSquaredError=%g"%testResults.rootMeanSquaredError)

In [ ]:
print ("meanAbsoluteError=%g"%testResults.meanAbsoluteError)

## Underfitting !
- decrease regularization parameter? 
- Add more features? 
- ? 

### Anyway let's get some insights from our data !

In [ ]:
data.show()

In [ ]:
data.head()

In [ ]:
data.printSchema()

In [ ]:
insights = lrModel.evaluate(data)

In [ ]:
pred = insights.predictions

In [ ]:
pred.take(1)

In [ ]:
pred_res = pred.withColumn('res_abs', abs(pred.prediction-pred.demand))

In [ ]:
#for item in pred_res.take(1)[0]: 
#  print (item)


In [ ]:
pred_res.take(1)

- Let's study the residual distrubution over the daily timesheet

In [ ]:

from pyspark.sql.functions import avg, stddev, format_number

In [ ]:
from pyspark.sql.functions import format_number
pred_res.groupBy('hr').agg(format_number(avg('res_abs'), 2).alias('avg_abs_residual'), 
                           format_number(avg('demand'), 2).alias('avg_demand'), 
                           format_number(stddev('prediction'), 2).alias('stddev_prediction'), 
                           format_number(stddev('demand'), 2).alias('stddev_demand')
                          ).sort('hr').show()

In [ ]:
pred_res.groupBy('season').agg(format_number(avg('res_abs'), 2).alias('avg_abs_residual'), 
                           format_number(avg('demand'), 2).alias('avg_demand'), 
                           format_number(stddev('prediction'), 2).alias('stddev_prediction'), 
                           format_number(stddev('demand'), 2).alias('stddev_demand')
                          ).sort('season').show(n=25)

In [ ]:
pred_res.groupBy('weathersit').agg(format_number(avg('res_abs'), 2).alias('avg_abs_residual'), 
                           format_number(avg('demand'), 2).alias('avg_demand'), 
                           format_number(stddev('prediction'), 2).alias('stddev_prediction'), 
                           format_number(stddev('demand'), 2).alias('stddev_demand')
                          ).sort('weathersit').show()

In [ ]:
indexed_data.columns

In [ ]:
hrs = indexed_data.select('hr').distinct().collect()

In [ ]:
print(hrs)

In [ ]:
# Let's add dummy variables (hrs)
for item in hrs:
  col = 'is_'+str(item[0])
  indexed_data = indexed_data.withColumn(col, (data.hr==item[0]).cast('integer'))
  

In [ ]:
indexed_data.printSchema()

In [ ]:
indexed_data.show(n=5)

In [ ]:
indexed_data.take(1)

In [ ]:
cols = indexed_data.drop('dayOfWeek').drop('demand').drop('hr').columns

In [ ]:
cols

In [ ]:
vec = VectorAssembler(inputCols=cols, outputCol='features')

In [ ]:
inputData = vec.transform(indexed_data)

In [ ]:
inputData.show(n=3)

In [ ]:
inputData.head(1)

In [ ]:
train, test = inputData.randomSplit([0.7, 0.3])

In [ ]:
train.cache()

In [ ]:
test.cache()

In [ ]:
model = lr.fit(train)

In [ ]:
results = model.evaluate(test)

In [ ]:
print ("rootMeanSquaredError={}".format(results.rootMeanSquaredError))
print ("explainedVariance={}".format(results.explainedVariance))
print ("meanAbsoluteError=%g" %results.meanAbsoluteError)
print ("r2=%g" %results.r2)

## The model is still far from being acceptable.. Keep going on ;-)

In [ ]:
results.predictions.select(['features', 'demand', 'prediction']).show(5)

In [ ]:
unlabeledData = testData.select('features')

In [ ]:
newPredictions = lrModel.transform(unlabeledData)

In [ ]:
newPredictions.show()

In [ ]:
newPredictions.select(format_number('prediction', 2).alias('prediction')).show(truncate = False)